<a href="https://colab.research.google.com/github/isb-cgc/Community-Notebooks/blob/master/Notebooks/How_to_use_the_Reactome_BQ_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use the Reactome BigQuery dataset
Check out other notebooks at our [Community Notebooks Repository](https://github.com/isb-cgc/Community-Notebooks)!

- **Title:** How to use the Reactome BigQuery dataset
- **Author:** John Phan
- **Created:** 2021-09-13
- **Purpose:** Demonstrate basic usage of the Reactome BigQuery dataset
- **URL:** https://github.com/isb-cgc/Community-Notebooks/blob/master/Notebooks/How_to_use_the_Reactome_BQ_dataset.ipynb

This notebook demonstrates basic usage of the Reactome BigQuery dataset. Analysis of this dataset can provide a powerful tool for identifying pathways related to cancer biomarkers. 

The Reactome is an open-source, manually curated, and peer-reviewed pathway database. More information can be found here: https://reactome.org/. 



# Initialize Notebook Environment

Before running the analysis, we need to load dependencies, authenticate to BigQuery, and customize notebook parameters.

## Import Dependencies

In [ ]:
# GCP Libraries
from google.cloud import bigquery
from google.colab import auth

# Data Analytics
import numpy as np
from scipy import stats

## Authenticate

Before using BigQuery, we need to get authorization for access to BigQuery and the Google Cloud. For more information see ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html). Alternative authentication methods can be found [here](https://googleapis.dev/python/google-api-core/latest/auth.html).

In [ ]:
# if you're using Google Colab, authenticate to gcloud with the following
auth.authenticate_user()

# alternatively, use the gcloud SDK
#!gcloud auth application-default login

## Parameters

Customize the following parameters based on your notebook, execution environment, or project.

In [ ]:
# set the google project that will be billed for this notebook's computations
google_project = 'google-project' ## change me

## BigQuery Client

Create the BigQuery client.

In [ ]:
# Create a client to access the data within BigQuery
client = bigquery.Client(google_project)

# Identify all Reactome Pathways Related to Genes

We can join tables from the Reactome BigQuery dataset to identify all pathways related to our genes of interest. We first have to map the gene names to Uniprot IDs in the Reactome "physical entities" table. These can then be mapped to Reactome pathways. We further filter the physical entity to pathway evidence codes to retain only interactions that have "Traceable Author Statements" (TAS) rather than just "Inferred from Electronic Annotation" (IEA) to avoid evidence that have not been manually curated. 

We use the following genes to identify related pathways. These genes were identified in an ovarian cancer chemo-response study by [Bosquet et al](https://molecular-cancer.biomedcentral.com/articles/10.1186/s12943-016-0548-9).  

In [ ]:
# set parameters for query
genes = "'RHOT1','MYO7A','ZBTB10','MATK','ST18','RPS23','GCNT1','DROSHA','NUAK1','CCPG1',\
'PDGFD','KLRAP1','MTAP','RNF13','THBS1','MLX','FAP','TIMP3','PRSS1','SLC7A11',\
'OLFML3','RPS20','MCM5','POLE','STEAP4','LRRC8D','WBP1L','ENTPD5','SYNE1','DPT',\
'COPZ2','TRIO','PDPR'"

In [ ]:
# run query and put results in data frame
pathways = client.query(("""
  SELECT
    DISTINCT pathway.*

  FROM
    `isb-cgc-bq.reactome_versioned.pathway_v77` as pathway

  INNER JOIN `isb-cgc-bq.reactome_versioned.pe_to_pathway_v77` as pe2pathway
    -- link pathways to physical entities via intermediate table
    ON pathway.stable_id = pe2pathway.pathway_stable_id

  INNER JOIN `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe
    -- link pathways to physical entities
    ON pe2pathway.pe_stable_id = pe.stable_id

  WHERE
    -- filter by stronger evidence: "Traceable Author Statement" 
    pe2pathway.evidence_code = 'TAS'

    -- filter by pathways that are related to genes in list
    AND pe.name IN ({genes}) 

  ORDER BY pathway.name ASC
""").format(
  genes=genes
)).result().to_dataframe()

In [ ]:
# Display the data frame
pathways

,stable_id,url,name,species,lowest_level
0,R-HSA-176187,https://reactome.org/PathwayBrowser/#/R-HSA-17...,Activation of ATR in response to replication s...,Homo sapiens,True
1,R-HSA-72662,https://reactome.org/PathwayBrowser/#/R-HSA-72662,Activation of the mRNA upon binding of the cap...,Homo sapiens,False
2,R-HSA-68962,https://reactome.org/PathwayBrowser/#/R-HSA-68962,Activation of the pre-replicative complex,Homo sapiens,True
3,R-HSA-352230,https://reactome.org/PathwayBrowser/#/R-HSA-35...,Amino acid transport across the plasma membrane,Homo sapiens,True
4,R-HSA-446203,https://reactome.org/PathwayBrowser/#/R-HSA-44...,Asparagine N-linked glycosylation,Homo sapiens,False
...,...,...,...,...,...
148,R-HSA-192823,https://reactome.org/PathwayBrowser/#/R-HSA-19...,Viral mRNA Translation,Homo sapiens,True
149,R-HSA-2187338,https://reactome.org/PathwayBrowser/#/R-HSA-21...,Visual phototransduction,Homo sapiens,False
150,R-HSA-193704,https://reactome.org/PathwayBrowser/#/R-HSA-19...,p75 NTR receptor-mediated signalling,Homo sapiens,False
151,R-HSA-72312,https://reactome.org/PathwayBrowser/#/R-HSA-72312,rRNA processing,Homo sapiens,False


If we're only interested in the lowest level pathways, i.e., pathways that are not parents of other pathways in the hierarchy, we can filter by the "lowest_level" field in the pathways table.

In [ ]:
# run query and put results in data frame
lowest_pathways = client.query(("""
  SELECT
    DISTINCT pathway.*

  FROM
    `isb-cgc-bq.reactome_versioned.pathway_v77` as pathway

  INNER JOIN `isb-cgc-bq.reactome_versioned.pe_to_pathway_v77` as pe2pathway
    -- link pathways to physical entities via intermediate table
    ON pathway.stable_id = pe2pathway.pathway_stable_id

  INNER JOIN `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe
    -- link pathways to physical entities
    ON pe2pathway.pe_stable_id = pe.stable_id

  WHERE
    -- filter by stronger evidence: "Traceable Author Statement" 
    pe2pathway.evidence_code = 'TAS'

    -- filter by pathways that are related to genes in list
    AND pe.name IN ({genes})

    -- filter to include just lowest level pathways
    AND pathway.lowest_level = TRUE

  ORDER BY pathway.name ASC
""").format(
  genes=genes
)).result().to_dataframe()

In [ ]:
# display data frame
lowest_pathways

,stable_id,url,name,species,lowest_level
0,R-HSA-176187,https://reactome.org/PathwayBrowser/#/R-HSA-17...,Activation of ATR in response to replication s...,Homo sapiens,True
1,R-HSA-68962,https://reactome.org/PathwayBrowser/#/R-HSA-68962,Activation of the pre-replicative complex,Homo sapiens,True
2,R-HSA-352230,https://reactome.org/PathwayBrowser/#/R-HSA-35...,Amino acid transport across the plasma membrane,Homo sapiens,True
3,R-HSA-210991,https://reactome.org/PathwayBrowser/#/R-HSA-21...,Basigin interactions,Homo sapiens,True
4,R-HSA-9013148,https://reactome.org/PathwayBrowser/#/R-HSA-90...,CDC42 GTPase cycle,Homo sapiens,True
5,R-HSA-6811434,https://reactome.org/PathwayBrowser/#/R-HSA-68...,COPI-dependent Golgi-to-ER retrograde traffic,Homo sapiens,True
6,R-HSA-6807878,https://reactome.org/PathwayBrowser/#/R-HSA-68...,COPI-mediated anterograde transport,Homo sapiens,True
7,R-HSA-163765,https://reactome.org/PathwayBrowser/#/R-HSA-16...,ChREBP activates metabolic gene expression,Homo sapiens,True
8,R-HSA-418885,https://reactome.org/PathwayBrowser/#/R-HSA-41...,DCC mediated attractive signaling,Homo sapiens,True
9,R-HSA-68952,https://reactome.org/PathwayBrowser/#/R-HSA-68952,DNA replication initiation,Homo sapiens,True


# Pathway Enrichment Analysis
We can identify pathways that are "enriched" with the genes of interest. In other words, we can answer the question: given a set of interesting genes, which pathways contain those genes at a frequency higher than random chance? By calculating the probability that a number of target genes are contained in each pathway, we can identify pathways most likely to be related. 

To do this, we can use a **chi-squared** test to determine if there is a statistically significant difference between the expected frequency of genes in a pathway compared to the observed frequency. 


In [ ]:
# set query parameters
lowest_level = True # only show pathways at the lowest level

## Construct SQL Query

A single query can be used to calculate the chi-squared statistic for all pathways. This query is rather lengthy, but can be broken up into a series of named sub-queries. We step through each query below.

First, we write a query that simply gets a list of all genes in the Reactome physical entity table:  

In [ ]:
gene_list_query = """
  -- Table that contains a list of all distinct genes that map to Reactome
  -- physical entities 
  SELECT
    DISTINCT pe.uniprot_id

  FROM
    `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe

  WHERE
    -- filter by pathways that are related to genes in list
    pe.name IN ({genes})
""".format(
  genes=genes
).strip("\n")

In [ ]:
print(gene_list_query)

  -- Table that contains a list of all distinct genes that map to Reactome
  -- physical entities 
  SELECT
    DISTINCT pe.uniprot_id

  FROM
    `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe

  WHERE
    -- filter by pathways that are related to genes in list
    pe.name IN ('RHOT1','MYO7A','ZBTB10','MATK','ST18','RPS23','GCNT1','DROSHA','NUAK1','CCPG1','PDGFD','KLRAP1','MTAP','RNF13','THBS1','MLX','FAP','TIMP3','PRSS1','SLC7A11','OLFML3','RPS20','MCM5','POLE','STEAP4','LRRC8D','WBP1L','ENTPD5','SYNE1','DPT','COPZ2','TRIO','PDPR')


We then create a query that counts all targets associated with each Reactome pathway. This query depends on the previous `gene_list_query` sub-query.  

In [ ]:
gene_pp_query = """
  -- Table that maps pathways to the total number of interesting genes within
  -- that pathway
  SELECT
    COUNT(DISTINCT gene_list_query.uniprot_id) as num_genes,
    pathway.stable_id,
    pathway.name

  FROM
    gene_list_query

  INNER JOIN `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe
    -- filter for interactions with genes that match a reactome
    -- physical entity
    ON gene_list_query.uniprot_id = pe.uniprot_id

  INNER JOIN `isb-cgc-bq.reactome_versioned.pe_to_pathway_v77` AS pe2pathway
    -- link physical entities to pathways via intermediate table
    ON pe.stable_id = pe2pathway.pe_stable_id

  INNER JOIN `isb-cgc-bq.reactome_versioned.pathway_v77` AS pathway
    -- link physical entities to pathways
    ON pe2pathway.pathway_stable_id = pathway.stable_id

  WHERE
    -- filter by stronger evidence: "Traceable Author Statement" 
    pe2pathway.evidence_code = 'TAS'

  GROUP BY pathway.stable_id, pathway.name
  ORDER BY num_genes DESC
""".strip("\n")

In [ ]:
print(gene_pp_query)

  -- Table that maps pathways to the total number of interesting genes within
  -- that pathway
  SELECT
    COUNT(DISTINCT gene_list_query.uniprot_id) as num_genes,
    pathway.stable_id,
    pathway.name

  FROM
    gene_list_query

  INNER JOIN `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe
    -- filter for interactions with genes that match a reactome
    -- physical entity
    ON gene_list_query.uniprot_id = pe.uniprot_id

  INNER JOIN `isb-cgc-bq.reactome_versioned.pe_to_pathway_v77` AS pe2pathway
    -- link physical entities to pathways via intermediate table
    ON pe.stable_id = pe2pathway.pe_stable_id

  INNER JOIN `isb-cgc-bq.reactome_versioned.pathway_v77` AS pathway
    -- link physical entities to pathways
    ON pe2pathway.pathway_stable_id = pathway.stable_id

  WHERE
    -- filter by stronger evidence: "Traceable Author Statement" 
    pe2pathway.evidence_code = 'TAS'

  GROUP BY pathway.stable_id, pathway.name
  ORDER BY num_genes DESC


Now we construct the same queries for genes that are NOT in the interesting gene list. These are prefixed with "not_gene". This query depends on the previous `gene_list_query` sub-query. 

In [ ]:
not_gene_list_query = """
  -- Table that contains a list of all genes that are NOT in the interest list
  -- This query depends on the previous "gene_list_query" sub-query.
  SELECT
    DISTINCT pe.uniprot_id

  FROM `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe

  WHERE
    pe.uniprot_id NOT IN (
      SELECT uniprot_id FROM gene_list_query
    )
""".strip("\n")

In [ ]:
print(not_gene_list_query)

  -- Table that contains a list of all genes that are NOT in the interest list
  -- This query depends on the previous "gene_list_query" sub-query.
  SELECT
    DISTINCT pe.uniprot_id

  FROM `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe

  WHERE
    pe.uniprot_id NOT IN (
      SELECT uniprot_id FROM gene_list_query
    )


In [ ]:
not_gene_pp_query = """
  -- Table that maps pathways to the number of proteins that are NOT drug
  -- targets in that pathway.
  SELECT
    COUNT(DISTINCT not_gene_list_query.uniprot_id) AS num_not_genes,
    pathway.stable_id,
    pathway.name

  FROM not_gene_list_query

  INNER JOIN `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe
    ON not_gene_list_query.uniprot_id = pe.uniprot_id

  INNER JOIN `isb-cgc-bq.reactome_versioned.pe_to_pathway_v77` AS pe2pathway
    ON pe.stable_id = pe2pathway.pe_stable_id
  
  INNER JOIN `isb-cgc-bq.reactome_versioned.pathway_v77` AS pathway
    ON pe2pathway.pathway_stable_id = pathway.stable_id

  WHERE
    -- filter by stronger evidence: "Traceable Author Statement" 
    pe2pathway.evidence_code = 'TAS'

  GROUP BY pathway.stable_id, pathway.name
  ORDER BY num_not_genes DESC
""".strip("\n")

In [ ]:
print(not_gene_pp_query)

  -- Table that maps pathways to the number of proteins that are NOT drug
  -- targets in that pathway.
  SELECT
    COUNT(DISTINCT not_gene_list_query.uniprot_id) AS num_not_genes,
    pathway.stable_id,
    pathway.name

  FROM not_gene_list_query

  INNER JOIN `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe
    ON not_gene_list_query.uniprot_id = pe.uniprot_id

  INNER JOIN `isb-cgc-bq.reactome_versioned.pe_to_pathway_v77` AS pe2pathway
    ON pe.stable_id = pe2pathway.pe_stable_id
  
  INNER JOIN `isb-cgc-bq.reactome_versioned.pathway_v77` AS pathway
    ON pe2pathway.pathway_stable_id = pathway.stable_id

  WHERE
    -- filter by stronger evidence: "Traceable Author Statement" 
    pe2pathway.evidence_code = 'TAS'

  GROUP BY pathway.stable_id, pathway.name
  ORDER BY num_not_genes DESC


For convenience, we create a sub-query to get the counts of the number of genes that are in the list and the number of genes that are not in the list.

In [ ]:
gene_count_query = """
  -- Table that contains the counts of # of genes that are/are not targets
  SELECT
    gene_count,
    not_gene_count,
    gene_count + not_gene_count AS total_count

  FROM 
    (SELECT COUNT(*) AS gene_count FROM gene_list_query),
    (SELECT COUNT(*) AS not_gene_count FROM not_gene_list_query)
""".strip("\n")

Now we can create more interesting queries for the contingency matrices that contain the observed and expected values. 

First, the observed contingency table counts for each pathway:

In [ ]:
observed_query = """
  -- Table with observed values per pathway in the contingency matrix
  SELECT
    gene_pp_query.num_genes AS in_gene_in_pathway,
    not_gene_pp_query.num_not_genes AS not_gene_in_pathway,
    gene_count_query.gene_count - gene_pp_query.num_genes AS in_gene_not_pathway,
    gene_count_query.not_gene_count - not_gene_pp_query.num_not_genes AS not_gene_not_pathway,
    gene_pp_query.stable_id,
    gene_pp_query.name

  FROM 
    gene_pp_query,
    gene_count_query

  INNER JOIN not_gene_pp_query
    ON gene_pp_query.stable_id = not_gene_pp_query.stable_id
""".strip("\n")

Then the observed row and column sums of the contingency table:

In [ ]:
sum_query = """
  -- Table with summed observed values per pathway in the contingency matrix
  SELECT
    observed_query.in_gene_in_pathway + observed_query.not_gene_in_pathway AS pathway_total,
    observed_query.in_gene_not_pathway + observed_query.not_gene_not_pathway AS not_pathway_total,
    observed_query.in_gene_in_pathway + observed_query.in_gene_not_pathway AS gene_total,
    observed_query.not_gene_in_pathway + observed_query.not_gene_not_pathway AS not_gene_total,
    observed_query.stable_id,
    observed_query.name

  FROM
    observed_query
""".strip("\n")

And the expected contingency table values for each pathway:

In [ ]:
expected_query = """  
  -- Table with the expected values per pathway in the contingency matrix
  SELECT 
    sum_query.gene_total * sum_query.pathway_total / gene_count_query.total_count AS exp_in_gene_in_pathway,
    sum_query.not_gene_total * sum_query.pathway_total / gene_count_query.total_count AS exp_not_gene_in_pathway,
    sum_query.gene_total * sum_query.not_pathway_total / gene_count_query.total_count AS exp_in_gene_not_pathway,
    sum_query.not_gene_total * sum_query.not_pathway_total / gene_count_query.total_count AS exp_not_gene_not_pathway,
    sum_query.stable_id,
    sum_query.name

  FROM 
    sum_query, gene_count_query
""".strip("\n")

Finally, we can calculate the chi-squared statistic for each pathway:

In [ ]:
chi_squared_query = """
  -- Table with the chi-squared statistic for each pathway
  SELECT
    -- Chi squared statistic with Yates' correction
    POW(ABS(observed_query.in_gene_in_pathway - expected_query.exp_in_gene_in_pathway) - 0.5, 2) / expected_query.exp_in_gene_in_pathway 
    + POW(ABS(observed_query.not_gene_in_pathway - expected_query.exp_not_gene_in_pathway) - 0.5, 2) / expected_query.exp_not_gene_in_pathway
    + POW(ABS(observed_query.in_gene_not_pathway - expected_query.exp_in_gene_not_pathway) - 0.5, 2) / expected_query.exp_in_gene_not_pathway
    + POW(ABS(observed_query.not_gene_not_pathway - expected_query.exp_not_gene_not_pathway) - 0.5, 2) / expected_query.exp_not_gene_not_pathway
    AS chi_squared_stat,
    observed_query.stable_id,
    observed_query.name

  FROM observed_query

  INNER JOIN expected_query
    ON observed_query.stable_id = expected_query.stable_id
""".strip("\n")

The final piece of the query optionally adds a filter that removes all pathways that are not at the lowest level of the hierarchy. This helps remove non-specific "pathways" such as the generic "Disease" pathway.

In [ ]:
lowest_level_filter = """
  INNER JOIN `isb-cgc-bq.reactome_versioned.pathway_v77` AS pathway
    ON chi_squared_query.stable_id = pathway.stable_id

  WHERE pathway.lowest_level = TRUE
""".strip("\n")

In [ ]:
print(lowest_level_filter)

  INNER JOIN `isb-cgc-bq.reactome_versioned.pathway_v77` AS pathway
    ON chi_squared_query.stable_id = pathway.stable_id

  WHERE pathway.lowest_level = TRUE


Now we can combine all sub-queries to create the final query:

In [ ]:
final_query = """
  WITH
    gene_list_query AS (
      {gene_list_query}
    ),

    gene_pp_query AS (
      {gene_pp_query}
    ),

    not_gene_list_query AS (
      {not_gene_list_query}
    ),

    not_gene_pp_query AS (
      {not_gene_pp_query}
    ),

    gene_count_query AS (
      {gene_count_query}
    ),

    observed_query AS (
      {observed_query}
    ),

    sum_query AS (
      {sum_query}
    ),

    expected_query AS (
      {expected_query}
    ),
    
    chi_squared_query AS (
      {chi_squared_query}
    )

  SELECT
    observed_query.in_gene_in_pathway,
    observed_query.in_gene_not_pathway,
    observed_query.not_gene_in_pathway,
    observed_query.not_gene_not_pathway,
    chi_squared_query.chi_squared_stat,
    chi_squared_query.stable_id,
    chi_squared_query.name

  FROM chi_squared_query

  INNER JOIN observed_query
    ON chi_squared_query.stable_id = observed_query.stable_id
  {lowest_level_filter}
  ORDER BY chi_squared_stat DESC
""".format(
  # make final query a little easier to read by removing/adding some white space 
  gene_list_query="\n    ".join(gene_list_query.strip().splitlines()),
  gene_pp_query="\n    ".join(gene_pp_query.strip().splitlines()),
  not_gene_list_query="\n    ".join(not_gene_list_query.strip().splitlines()),
  not_gene_pp_query="\n    ".join(not_gene_pp_query.strip().splitlines()),
  gene_count_query="\n    ".join(gene_count_query.strip().splitlines()),
  observed_query="\n    ".join(observed_query.strip().splitlines()),
  sum_query="\n    ".join(sum_query.strip().splitlines()),
  expected_query="\n    ".join(expected_query.strip().splitlines()),
  chi_squared_query="\n    ".join(chi_squared_query.strip().splitlines()),
  lowest_level_filter=(
    "\n  "+"\n".join(lowest_level_filter.strip().splitlines())+"\n" if lowest_level else ""
  )
).strip("\n")

## Display Final Query

In [ ]:
# print the formatted final query
print(final_query)

  WITH
    gene_list_query AS (
      -- Table that contains a list of all distinct genes that map to Reactome
      -- physical entities 
      SELECT
        DISTINCT pe.uniprot_id
    
      FROM
        `isb-cgc-bq.reactome_versioned.physical_entity_v77` AS pe
    
      WHERE
        -- filter by pathways that are related to genes in list
        pe.name IN ('RHOT1','MYO7A','ZBTB10','MATK','ST18','RPS23','GCNT1','DROSHA','NUAK1','CCPG1','PDGFD','KLRAP1','MTAP','RNF13','THBS1','MLX','FAP','TIMP3','PRSS1','SLC7A11','OLFML3','RPS20','MCM5','POLE','STEAP4','LRRC8D','WBP1L','ENTPD5','SYNE1','DPT','COPZ2','TRIO','PDPR')
    ),

    gene_pp_query AS (
      -- Table that maps pathways to the total number of interesting genes within
      -- that pathway
      SELECT
        COUNT(DISTINCT gene_list_query.uniprot_id) as num_genes,
        pathway.stable_id,
        pathway.name
    
      FROM
        gene_list_query
    
      INNER JOIN `isb-cgc-bq.reactome_versioned.physical_entity_v77

## Execute the Query

Now execute the query to calculate a chi-squared statistic for each pathway:

In [ ]:
# run query and put results in data frame
chi_squared_pathways = client.query(final_query).result().to_dataframe()

In [ ]:
# display the data frame
chi_squared_pathways

,in_gene_in_pathway,in_gene_not_pathway,not_gene_in_pathway,not_gene_not_pathway,chi_squared_stat,stable_id,name
0,2,19,31,11114,33.477023,R-HSA-68962,Activation of the pre-replicative complex
1,1,20,3,11142,32.311989,R-HSA-1237112,Methionine salvage pathway
2,1,20,3,11142,32.311989,R-HSA-9013425,RHOT1 GTPase cycle
3,2,19,50,11095,20.236790,R-HSA-72695,"Formation of the ternary complex, and subseque..."
4,1,20,6,11139,18.048197,R-HSA-163765,ChREBP activates metabolic gene expression
5,2,19,57,11088,17.513505,R-HSA-72649,Translation initiation complex formation
6,2,19,57,11088,17.513505,R-HSA-72702,Ribosomal scanning and start codon recognition
7,1,20,7,11138,15.671798,R-HSA-8850843,Phosphate bond hydrolysis by NTPDase proteins
8,1,20,7,11138,15.671798,R-HSA-68952,DNA replication initiation
9,1,20,10,11135,11.137000,R-HSA-418885,DCC mediated attractive signaling


## Calculate P-Values

BigQuery does not have statistical functions to calculate p-values, so we use the SciPy stats library and update the data frame with a new p-value column:

In [ ]:
chi_squared_pathways['p_value'] = 1-stats.chi2.cdf(chi_squared_pathways['chi_squared_stat'], 1)
chi_squared_pathways

,in_gene_in_pathway,in_gene_not_pathway,not_gene_in_pathway,not_gene_not_pathway,chi_squared_stat,stable_id,name,p_value
0,2,19,31,11114,33.477023,R-HSA-68962,Activation of the pre-replicative complex,7.211088e-09
1,1,20,3,11142,32.311989,R-HSA-1237112,Methionine salvage pathway,1.313007e-08
2,1,20,3,11142,32.311989,R-HSA-9013425,RHOT1 GTPase cycle,1.313007e-08
3,2,19,50,11095,20.236790,R-HSA-72695,"Formation of the ternary complex, and subseque...",6.842431e-06
4,1,20,6,11139,18.048197,R-HSA-163765,ChREBP activates metabolic gene expression,2.153825e-05
5,2,19,57,11088,17.513505,R-HSA-72649,Translation initiation complex formation,2.852741e-05
6,2,19,57,11088,17.513505,R-HSA-72702,Ribosomal scanning and start codon recognition,2.852741e-05
7,1,20,7,11138,15.671798,R-HSA-8850843,Phosphate bond hydrolysis by NTPDase proteins,7.533920e-05
8,1,20,7,11138,15.671798,R-HSA-68952,DNA replication initiation,7.533920e-05
9,1,20,10,11135,11.137000,R-HSA-418885,DCC mediated attractive signaling,8.462266e-04


## Adjust for Multiple Testing

Since we're testing multiple pathways, we need to adjust the p-value threshold for significance accordingly. The number of pathways tested depends on whether or not we're considering all pathways, or just the lowest level pathways. That count can be obtained with the following query.

In [ ]:
# run query and put results in data frame
num_pathways_result = client.query(("""
  SELECT
    COUNT (*) AS num_pathways
  FROM
    `isb-cgc-bq.reactome_versioned.pathway_v77` as pathway
  {lowest_level_filter}
""").format(
  lowest_level_filter=("WHERE lowest_level = TRUE" if lowest_level else "")
)).result().to_dataframe()

In [ ]:
# display data frame
num_pathways_result

,num_pathways
0,1773


In [ ]:
# adjust significance threshold for multiple testing, using a p-value of 0.01
num_pathways = num_pathways_result['num_pathways'][0]
significance_threshold = 0.01/num_pathways
print('Significance Threshold: {}'.format(significance_threshold))

# find all pathways that meet the significance criterion after adjusting for
# multiple testing
significant_pathway_index = chi_squared_pathways['p_value']<significance_threshold

# list of significant pathways
significant_pathways = chi_squared_pathways[significant_pathway_index]

Significance Threshold: 5.640157924421884e-06


The final result is a list of all pathways in which the targeted proteins are enriched, or over-represented, at a rate higher than random chance. 

In [ ]:
# display the final data frame
significant_pathways

,in_gene_in_pathway,in_gene_not_pathway,not_gene_in_pathway,not_gene_not_pathway,chi_squared_stat,stable_id,name,p_value
0,2,19,31,11114,33.477023,R-HSA-68962,Activation of the pre-replicative complex,7.211088e-09
1,1,20,3,11142,32.311989,R-HSA-1237112,Methionine salvage pathway,1.313007e-08
2,1,20,3,11142,32.311989,R-HSA-9013425,RHOT1 GTPase cycle,1.313007e-08


The results of this analysis suggest that at least three pathways may be related to the genes identified.  

## Verify Results by Comparing to SciPy Chi-Squared Function

We verify these BigQuery results by calculating the same chi-squared statistic using the SciPy package. Comparing the SciPy p-values and statistics to those of the BigQuery-derived results confirms that they are identical.

In [ ]:
# extract observed values from bigquery result
observed = chi_squared_pathways[[
  'in_gene_in_pathway',
  'in_gene_not_pathway',
  'not_gene_in_pathway',
  'not_gene_not_pathway'
]]

# calculate the chi-squared statistic using the scipy stats package  
chi2_stat = []
chi2_pvalue = []
for index, row in observed.iterrows():
  stat, pvalue, _, _ = stats.chi2_contingency(
    np.reshape(np.matrix(row), (2,2)), correction=True
  )
  chi2_stat.append(stat)
  chi2_pvalue.append(pvalue)

# add columns to the original data frame
chi_squared_pathways['scipy_stat'] = chi2_stat
chi_squared_pathways['scipy_p_value'] = chi2_pvalue

In [ ]:
# display the updated data frame
chi_squared_pathways

,in_gene_in_pathway,in_gene_not_pathway,not_gene_in_pathway,not_gene_not_pathway,chi_squared_stat,stable_id,name,p_value,scipy_stat,scipy_p_value
0,2,19,31,11114,33.477023,R-HSA-68962,Activation of the pre-replicative complex,7.211088e-09,33.477023,7.211088e-09
1,1,20,3,11142,32.311989,R-HSA-1237112,Methionine salvage pathway,1.313007e-08,32.311989,1.313007e-08
2,1,20,3,11142,32.311989,R-HSA-9013425,RHOT1 GTPase cycle,1.313007e-08,32.311989,1.313007e-08
3,2,19,50,11095,20.236790,R-HSA-72695,"Formation of the ternary complex, and subseque...",6.842431e-06,20.236790,6.842431e-06
4,1,20,6,11139,18.048197,R-HSA-163765,ChREBP activates metabolic gene expression,2.153825e-05,18.048197,2.153825e-05
5,2,19,57,11088,17.513505,R-HSA-72649,Translation initiation complex formation,2.852741e-05,17.513505,2.852741e-05
6,2,19,57,11088,17.513505,R-HSA-72702,Ribosomal scanning and start codon recognition,2.852741e-05,17.513505,2.852741e-05
7,1,20,7,11138,15.671798,R-HSA-8850843,Phosphate bond hydrolysis by NTPDase proteins,7.533920e-05,15.671798,7.533920e-05
8,1,20,7,11138,15.671798,R-HSA-68952,DNA replication initiation,7.533920e-05,15.671798,7.533920e-05
9,1,20,10,11135,11.137000,R-HSA-418885,DCC mediated attractive signaling,8.462266e-04,11.137000,8.462266e-04


# Conclusion

This notebook demonstrated usage of the Reactome BigQuery dataset for basic cancer pathway identification from a gene set, as well as a more complex pathway enrichment analysis using a chi-squared statistic.